In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import (Dense, 
                                     BatchNormalization, 
                                     LeakyReLU, 
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten)

import tensorflow_addons as tfa

from tensorflow.keras.applications import EfficientNetB3
from keras.models import Model
from keras.layers import Activation
import efficientnet.tfkeras
import efficientnet.keras as efn 

from keras.utils.vis_utils import plot_model

In [43]:
def split_model(model, start, end, name):
    """Splits the EfficientNet model based on a range defined by the start and end parameters. Allows all the features of the model remain 
        while only keeping a small portion. The previous implementations relied on manually separating the layers, possibly reducing
        performance"""    

    structure = model.get_config()
    layers_used = set()
    for i, l in enumerate(structure['layers']):
        if i == 0:
            structure['layers'][0]['config']['batch_input_shape'] = model.layers[start].input_shape
            if i != start:
                structure['layers'][0]['name']
                structure['layers'][0]['config']['name'] = structure['layers'][0]['name']
        elif i < start or i > end:
            continue
        layers_used.add(l['name'])

    #tf.print(structure['layers'][0]['config']['batch_input_shape']) #(64,64,144)
    #tf.print(structure)
    #tf.print(layers_used)

    # extract layers to be used 
    layers = [l for l in structure['layers'] if l['name'] in layers_used]
    layers[1]['inbound_nodes'][0][0][0] = layers[0]['name']

    #tf.print(layers)
    #tf.print(layers[1]['inbound_nodes'][0][0][0])

    # set model structure
    structure['layers'] = layers
    structure['input_layers'][0][0] = layers[0]['name']
    structure['output_layers'][0][0] = layers[-1]['name']

    #tf.print(structure['layers'])
    #tf.print(structure['input_layers'][0][0])
    #tf.print(structure['output_layers'][0][0])

    # create new model
    modelsplit = Model.from_config(structure)
    for l in modelsplit.layers:
        orig_l = model.get_layer(l.name)
        if orig_l is not None:
            l.set_weights(orig_l.get_weights())
    
    tf.print(modelsplit)
    """
    modelsplit.compile()
    modelsplit.summary()

    """
    for layer in modelsplit.layers:
        layer._name = layer.name + str(name)

    modelsplit._name = 'EfficientNet_' + str(name)

    modelsplit.compile()
    modelsplit.summary()
    
    return modelsplit

In [44]:
model = efn.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(256, 256, 3))

split_model(model, 30, 71, 'name')

Model: "EfficientNet_name"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21name (InputLayer)       [(None, 64, 64, 144) 0                                            
__________________________________________________________________________________________________
block2a_bnname (BatchNormalizat (None, 64, 64, 144)  576         input_21name[0][0]               
__________________________________________________________________________________________________
block2a_activationname (Activat (None, 64, 64, 144)  0           block2a_bnname[0][0]             
__________________________________________________________________________________________________
block2a_se_squeezename (GlobalA (None, 144)          0           block2a_activationname[0][0]     
__________________________________________________________________________________